### Colab에서 실행한다고 가정하고 작성함.

In [ ]:
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### openai 설치 안된 경우 설치

In [ ]:
!pip install openai

### 사용자 API키, 사용자 경로 수정한 후 실행할 것! 

In [13]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import torch
from openai import OpenAI
import torch.nn.functional as F
import pickle

# OpenAI 클라이언트 설정
client = OpenAI(
    api_key="사용자 API키",
    base_url="https://api.upstage.ai/v1/solar"
)

# 데이터 로딩
df = pd.read_excel('사용자 경로')

# 결측값 처리
df['DATA'] = df['DATA'].fillna('')

# label이 1인 데이터 필터링
filtered_df = df[df['TARGET'] == 1]

# 문장과 메뉴 이름을 리스트로 변환
texts = filtered_df['DATA'].tolist()
menu_names = filtered_df['MENU_NAME'].tolist()  # MENU_NAME 컬럼 추가

# OpenAI 임베딩 생성 함수
def get_embeddings(texts, batch_size=100):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        response = client.embeddings.create(
            input=batch_texts,
            model="solar-embedding-1-large-query"
        )
        batch_embeddings = [np.array(embedding.embedding) for embedding in response.data]
        embeddings.extend(batch_embeddings)
    return embeddings

# 필터링된 데이터의 임베딩 생성
embeddings = get_embeddings(texts)
answer = {}
for idx in range(len(embeddings)) :
  key = menu_names[idx].strip()
  value = [embeddings[idx], texts[idx]]
  answer[key] = value

# 임베딩을 피클 파일로 저장
# Google Drive의 특정 경로에 'embeddings.pkl'로 저장
save_path = '특정 경로'
with open(save_path, 'wb') as f:
    pickle.dump(answer, f)
